# Cuaderno de creación de lista negra de usuarios

El código definido a continuación tiene como objetivo analizar las publicaciones de la carpeta [`datos`](./datos), y crear una lista de usuarios que no deberían ser considerados en el análisis de datos.

Se consideraran para esta lista usuarios que sean autores de: 

* Publicaciones hechas para fomentar el cuidado de la salud mental
    * Si bien se pueden encontrar palabras clave que indiquen tendencias suicidas, no necesariamente reflejan problemas mentales en quien las publica
* Publicaciones repetidas (spam, bots, ofertas de trabajo, etc.)
* Publicaciones de páginas de noticias

Además, es posible excluir tweets que se consideren irrelevantes para el análisis, como por ejemplo:

* Aquellos que usen una de las palabras claves del conjunto de datos, pero que no tengan relación con el tema
    * Ejemplo: "Mi hermano suele colgarme cuando le llamo"

<hr></hr>

### Sobre los datos

Los datos procesados aquí provienen, en su mayoría, de Twitter, y fueron obtenidos usando una serie de palabras/frases clave por parte del equipo.

Además, se procesará un conjunto de textos clasificados como "con riesgo suicida" y "sin riesgo suicida". Estos textos se obtuvieron de un repositorio en línea por el director del proyecto, y no se procesan en un notebook adicional debido a que aquí ya se encuentra la funcionalidad necesaria para procesarlos.

Para ejecutar las siguientes instrucciones, se empleará el lenguaje de R.

Será de utilidad en el análisis, además, los paquetes `tm` y `SnowballC`, que permiten realizar análisis sobre texto y lematización.

In [1]:
# Se cargan las librerías
library('tm')
library('dplyr')
library('SnowballC')
library('stringr')

Warning message:
"package 'tm' was built under R version 4.2.3"
Loading required package: NLP

Warning message:
"package 'dplyr' was built under R version 4.2.3"

Attaching package: 'dplyr'


The following objects are masked from 'package:stats':

    filter, lag


The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union


Warning message:
"package 'SnowballC' was built under R version 4.2.3"
Warning message:
"package 'stringr' was built under R version 4.2.3"


Primero que nada, definimos funciones que nos permitan manipular qué archivos queremos analizar.

Estas funciones permiten procesar de manera genérica los archivos de la carpeta [`datos`](./datos), de tal forma que se contemplen únicamente los registros de usuarios con más de N publicaciones.

Otras consideraciones sobre los datos serán más específicas y se definirán más adelante.

In [2]:
# Definimos una lista de archivos ya procesados, de manera que no se carguen todos en cada sesión
archivos_procesados <- readLines('./datos/lista_procesados.txt')

In [3]:
# Agregamos además, functiones que nos permitan actualizar la lista de archivos procesados

guardar_lista <- function(ruta){
    writeLines(archivos_procesados, ruta)
}


actualizar_lista <- function(archivos, reemplazar = FALSE, guardar = FALSE){

    if (reemplazar) {
        archivos_procesados <- archivos
    }
    else {
        archivos_procesados <- c(archivos_procesados, archivos)
        archivos_procesados <- unique(archivos_procesados)
    }

    if (guardar) {
        writeLines(archivos_procesados, './datos/lista_procesados.txt')
    }

}

actualizar_lista_indicar_procesados <- function(archivos_procesando){

    archivos_procesados_nuevos <- c()
    
    for (archivo in archivos_procesando) {

        # Solicitar confirmación
        respuesta <- readline(paste0("¿Desea indicar el archivo [", archivo, "] como procesado? (s/n)"))

        if (tolower(respuesta) == 's') {
            archivos_procesados_nuevos <- c(archivos_procesados_nuevos, archivo)
        }
    }

    respuesta <- readline("Lista actualizada. ¿Desea guardar la lista? (s/n)")

    guardar <- FALSE

    if (tolower(respuesta) == 's') {
        guardar <- TRUE
    }
    
    actualizar_lista(archivos_procesados_nuevos, reemplazar = FALSE, guardar = guardar)
}


Para optimizar el proceso de lectura de archivos, vamos a definir funciones que:

* Descubran los archivos .csv dada una serie de carpetas base.
* Extraigan el nombre de los mismos para renombrarlos de ser necesario.
* Lean los archivos .csv y los conviertan en un dataframe de R.

In [4]:
# Definimos una función que nos permita conocer las rutas de los archivos a procesar, dada una lista de carpetas base
obtener_rutas <- function(carpetas) {

    rutas <- c()

    for (carpeta in carpetas) {

        rutas <- c(
            rutas, # Rutas anteriores
            list.files(carpeta, pattern = "*.csv", full.names = TRUE) # Rutas de los archivos descubiertos
        )
    }

    return(rutas)
}

# Definimos una función que nos de el nombre del archivo a partir de su ruta
obtener_nombre_archivo <- function(ruta) {

    # Separamos la ruta por las diagonales
    ruta_separada <- strsplit(ruta, "/", fixed = TRUE)[[1]]

    # Obtenemos el último elemento de la ruta
    nombre_archivo <- ruta_separada[length(ruta_separada)]

    return(nombre_archivo)
}

Ahora, es momento de realizar una estrategia de lectura de archivos. Debido a que el volumen de tweets suministrados es grande, se optará por leer los archivos uno a uno.

A continuación, únicamente relacionaremos el la ruta del archivo con su nombre, para leerlo en el momento que sea necesario y renombrar el archivo de salida de forma correspondiente.

In [5]:
obtener_archivos_por_leer <- function(carpetas, archivos_procesados, verbose){

    # Obtenemos las rutas de los archivos a procesar
    rutas <- obtener_rutas(carpetas)

    # Se juntan las rutas con los nombres de los archivos

    archivos_sin_leer <- list()

    for (ruta in rutas){

        nombre_archivo <- obtener_nombre_archivo(ruta)

        if (!(nombre_archivo %in% archivos_procesados)) {
            
            # Agregamos el nombre del archivo y su ruta a la lista
            archivos_sin_leer <- append(
                archivos_sin_leer,
                list( # El método append toma las dos listas y las junta en una sola
                    list( # Por eso, la lista contiene otra lista, que es una tupla
                        nombre = nombre_archivo,
                        ruta = ruta
                    )
                )
            )
            
        }

    }

    if (verbose) { # Si se solicita, imprimimos los archivos encontrados

        if (length(archivos_sin_leer) == 0){
            print('No se encontraron archivos nuevos.')
        } else {
            print('Se encontraron los siguientes archivos:')
        }
        
        for (archivo in archivos_sin_leer) {
            print(archivo$nombre)
        }
    }

    return(archivos_sin_leer)
}

# Está función de escritura mantiene el formato en el que originalmente se encuentan los .csv con Tweets
escribir_dataframe <- function(dataframe, ruta_archivo) {
    write.csv(
        dataframe,
        file = ruta_archivo,
        row.names = FALSE,
        col.names = TRUE,
        quote = TRUE
    )
}

Ahora, empleamos las funciones para obtener los archivos que leeremos.

In [6]:
# Leeremos los archivos .csv de las carpetas especificadas
# Hecho de esta manera, emplearemos las rutas para procesar los archivos
carpetas_a_explorar <- c(
    './datos/twitter/'
)

rutas_archivos <- obtener_archivos_por_leer(carpetas_a_explorar, archivos_procesados, verbose = TRUE)

[1] "Se encontraron los siguientes archivos:"
[1] "matarme_complete.csv"


In [7]:
# Guardamos los archivos que se van a procesar en una lista

archivos_de_la_sesion <- c()

for (archivo in rutas_archivos) {
    archivos_de_la_sesion <- c(archivos_de_la_sesion, archivo$nombre)
}

Después, reduciremos el tamaño de los conjuntos de datos, y mantendremos sólo aquellos usuarios con N publicaciones o más.

Esto nos permitirá hacer una revisión manual después, para detectar posibles usuarios que no deberían ser considerados en la construcción de los conjuntos de datos limpios.

In [35]:
# Obtenemos del dataframe las filas que tengan usuarios que aparezcan más de N veces

obtener_publicaciones <- function(dataframe, pub_minimas) {

    dataframe <- dataframe %>% 
        group_by(User) %>% 
        filter(n() >= pub_minimas)

    return(dataframe)
}

reducir_archivos <- function(archivos, pub_minimas, excepciones_archivos_a_reducir, ruta_guardado, verbose, sufijo) {

    
    for (ruta_nombre in archivos){

        ruta <- ruta_nombre$ruta
        nombre <- ruta_nombre$nombre

        if (!(nombre %in% excepciones_archivos_a_reducir)){

            # Leemos el archivo

            if (verbose){
                print(paste('Leyendo el archivo', nombre))
            }

            dataframe <- read.csv(file = ruta)

            filas_iniciales <- nrow(dataframe)

            # Obtenemos las publicaciones de los usuarios que aparezcan más de N veces

            if (verbose){
                print(paste('Filtrando usuarios con menos de', pub_minimas, 'publicaciones'))
            }

            dataframe <- obtener_publicaciones(dataframe, numero_min_publicaciones)

            filas_finales <- nrow(dataframe)

            # Creamos la nueva ruta. Le quitamos el .csv al nombre del archivo
            nueva_ruta <- paste0(ruta_guardado, substr(nombre, 1, nchar(nombre) - 4), sufijo)

            # Escribimos el archivo
            escribir_dataframe(dataframe, nueva_ruta)

            if (verbose){
                print(
                    paste('El archivo',
                            nombre,
                            '(# ',
                            filas_iniciales ,
                            'líneas) se redujo ( ahora',
                            obtener_nombre_archivo(nueva_ruta),
                            ', #',
                            filas_finales,
                            ' líneas).'
                    )
                )
            }

        }
        else if (verbose){
            print(paste('El archivo', nombre, 'no se redujo.'))
        }

    }
}

Ahora que ya definimos este paso del pipeline, es momento de ejecutarlo.

Toma en cuenta que necesitas crear el directorio en el que vas a guardar los archivos de salida.

In [56]:
# Si lo deseamos, podemos indicar archivos que no queremos pasar por el proceso de reducción
excepciones_archivos_a_reducir <- c(
    
)

numero_min_publicaciones <- 10

reducir_archivos(
    rutas_archivos,
    numero_min_publicaciones, 
    excepciones_archivos_a_reducir,
    ruta_guardado = './datos_limpios/datos_por_revisar/twitter/',
    verbose = TRUE,
    sufijo = '_reducido.csv'
)

[1] "Leyendo el archivo matarme_complete.csv"
[1] "Filtrando usuarios con menos de 10 publicaciones"


Warning message in write.csv(dataframe, file = ruta_archivo, row.names = FALSE, :
"attempt to set 'col.names' ignored"


[1] "El archivo matarme_complete.csv (#  3960745 líneas) se redujo ( ahora matarme_complete_reducido.csv , # 770485  líneas)."


In [9]:
# Guardar el progreso en la sesión
actualizar_lista_indicar_procesados(archivos_de_la_sesion)

Debido a que existen muchas palabras de poca relevancia y múltiples frases (a veces generadas a través de aplicaciones, otras son letras de canciones que usan la palabra con otro significado, y a veces incluso son etiquetas a usuarios populares o de influencia política), las celdas siguientes atienden necesidades específicas de la limpieza de cada conjunto de datos.

No obstante, por practicidad, se declaran previamente las funciones que se emplearán para limpiar los conjuntos de datos, con el fin de declararlas en la sesión más accesiblemente.

In [10]:
# Se juntan los vectores que contienen los nombres de los usuarios, para ser unificados en una sola lista negra
# Se le suministrarán varios vectores de usuarios, y se devolverá un vector con los usuarios únicos

juntar_usuarios <- function(listas_usuarios, ordenar = FALSE) {

    usuarios <- c()

    for (lista in listas_usuarios) {
        usuarios <- c(usuarios, lista)
    }

    usuarios <- unique(usuarios)

    if (ordenar) {
        usuarios <- sort(usuarios)
    }

    return(usuarios)
}

In [19]:
# Función que obtiene los tweets de un usuario y determina si todos contienen un texto clave
# de una lista dada

tweetsSonRepetitivos <- function(dataset, usuario, textosClave, ignorarSignosPunt = FALSE) {
    # Obtenemos los tweets del usuario
    tweets <- dataset %>%
        filter(User == usuario) %>%
        pull(Content) # pull() devuelve un vector con los valores de la columna
    
    # Revisamos si todos los tweets contienen alguno de los textos clave
    for (texto in textosClave){
        todosContienen <- all(grepl(texto, tweets, ignore.case = TRUE))

        # Nos conformamos con saber que uno de los textos que deseamos excluir está en los tweets,
        # pues es suficiente para determinar que el usuario repite contenido
        if (todosContienen) {
            return (TRUE)
        }

        if(ignorarSignosPunt) {
            # Si no todos los tweets contienen el texto clave, pero ignoramos los signos de puntuación,
            # entonces volvemos a revisar

            texto_sin_signos <- gsub("[[:punct:]]", "", texto)

            todosContienen <- all(grepl(texto_sin_signos, tweets, ignore.case = TRUE))
        }

    }
    
    return (todosContienen)
}

# Función que toma un dataset y una serie de textos clave, y devuelve los usuarios que tienen todos
# sus tweets con al menos uno de esos textos clave

obtenerUsuariosRepetitivos <- function(dataset, textosClave, ignorarSignosPunt = FALSE) {
    # Obtenemos los usuarios
    usuarios <- dataset %>%
        distinct(User) %>%
        pull(User)
    
    # Obtenemos los usuarios que tienen todos sus tweets con alguno de sus textos clave en este vector
    usuarios_repetitivos <- c()
    
    for (usuario in usuarios) {

        if (tweetsSonRepetitivos(dataset, usuario, textosClave, ignorarSignosPunt)) {
            usuarios_repetitivos <- c(usuarios_repetitivos, usuario)
        }
    }
    
    return (usuarios_repetitivos)
}

In [12]:
raiz_verbo_aparece <- function(texto, raices_a_comparar, lenguaje) {

    palabras <- unlist(strsplit(texto, " "))
    raices_palabras <- wordStem(palabras, language = lenguaje)
    
    for (raiz_palabra in raices_palabras) {
        
        for (raiz_verbo in raices_a_comparar) {
            
            if ( grepl(raiz_verbo, raiz_palabra, ignore.case = TRUE) ) {
                return(TRUE)
            }
        }

    }

    return(FALSE)

}

# Usando el paquete SnowballC, se buscarán los tweets que contengan alguna conjugación de los verbos indicados

excluir_tweets_segun_verbos <- function(dataframe, palabras, language) {

    # Se obtienen las raíces de las palabras
    raices_palabras <- wordStem(palabras, language = language)

    # Se obtienen los tweets que no contengan ninguna de las raíces de las palabras (i.e., el verbo no aparece
    # en ninguna conjugación)
    tweets_sin_palabras <- dataframe %>%
        mutate(
            contiene_raices_palabras = sapply(Content, raiz_verbo_aparece, raices_palabras, language)
        ) %>%
        filter(contiene_raices_palabras == FALSE) %>%
        select(-contiene_raices_palabras) # Se elimina la columna auxiliar

    return(tweets_sin_palabras)
}

excluir_tweets_segun_expresiones <- function(dataframe, expresiones) {

    # Se obtienen los tweets que no contengan ninguna de las expresiones
    tweets_sin_palabras <- dataframe %>%
        mutate(
            contiene_expresiones = as.vector(sapply(Content, comparar_tweet_con_palabras, expresiones))
        ) %>%
        filter(contiene_expresiones == FALSE) %>%
        select(-contiene_expresiones) # Se elimina la columna auxiliar

    return(tweets_sin_palabras)
}

comparar_tweet_con_palabras <- function(tweet, palabras) {

    # Se revisa si alguna de las palabras despreciables está en el tweet

    for (palabra in palabras) {

        if (grepl(palabra, tweet, ignore.case = TRUE)) {
            return(TRUE)
        }
    }

    return(FALSE)

}

# Función que devuelve los usuarios que usan al menos una de las palabras despreciables indicadas en sus tweets

obtener_usuarios_segun_palabras <- function(dataframe, palabras) {

    # Se obtienen los usuarios que usan al menos una de las palabras despreciables indicadas en sus tweets
    usuarios_con_palabras <- c()

    # Se recorren todos los tweets, y se revisa si alguno contiene alguna de las palabras despreciables,
    # para determinar si el usuario que lo escribió debe ser incluido en la lista negra

    usuarios_con_palabras <- dataframe %>%
        mutate(
            contiene_palabras = as.vector(sapply(Content, comparar_tweet_con_palabras, palabras))
        ) %>%
        filter(contiene_palabras == TRUE) %>%
        distinct(User) %>%
        pull(User)

    return (usuarios_con_palabras)

}

retirar_tweets_con_enlaces <- function(dataframe) {

    formatos <- c(
        'http://',
        'https://',
        'www.'
    )

    # Se obtienen los tweets que no contengan ninguna de las expresiones
    tweets_sin_enlaces <- excluir_tweets_segun_expresiones(dataframe, formatos)

    return(tweets_sin_enlaces)
}

In [13]:
ruta_lista_negra <- './datos_limpios/lista_negra_revisada.txt'

cargar_lista_negra <- function(ruta = ruta_lista_negra) {

    lista_negra <- readLines(ruta)

    return(lista_negra)
}

escribir_lista_negra <- function(lista_negra, ruta = ruta_lista_negra, retirar_duplicados = TRUE) {

    if(retirar_duplicados) {
        lista_negra <- unique(lista_negra)
    }
    
    writeLines(lista_negra, ruta)

}

anexar_a_lista <- function(usuarios, ruta = ruta_lista_negra) {

    # Por defecto anexa los usuarios a la lista negra

    conexion <- file(ruta, open = 'a')
    writeLines(usuarios, conexion)
    close(conexion)

}

ordenar_lista_negra <- function(ruta = ruta_lista_negra) {

    lista_negra <- cargar_lista_negra(ruta)

    lista_negra <- sort(lista_negra)

    escribir_lista_negra(lista_negra, ruta)

}

In [14]:
retirar_tweets_usuarios <- function(dataframe, usuarios) {
    return (
        dataframe %>%
            filter(!User %in% usuarios)
    )
}

### Datasets: `ahorcarme_complete.csv` y `colgarme_complete.csv`

In [ ]:
# Obtenemos los datos del archivo 'ahorcarme_complete.csv'
publicaciones_ahorcarme <- obtener_dataframe('ahorcarme_complete.csv', datos)
publicaciones_colgarme <- obtener_dataframe('colgarme_complete.csv', datos)

In [ ]:
# Filtramos los usuarios que tengan más de 5 publicaciones
publicaciones_ahorcarme_reducidas <- obtener_publicaciones(publicaciones_ahorcarme, 5)
publicaciones_colgarme_reducidas <- obtener_publicaciones(publicaciones_colgarme, 5)

In [ ]:
# Creamos la lista de los usuarios que aparecen más de 5 veces en las publicaciones

usuarios_ahorcarme <- publicaciones_ahorcarme_reducidas$User
usuarios_colgarme <- publicaciones_colgarme_reducidas$User

lista_negra <- juntar_usuarios(
    c(
        usuarios_ahorcarme,
        usuarios_colgarme
    )
)

In [ ]:
head(lista_negra)

Ahora, podemos revisar manualmente los tweets de los usuarios que colocamos en la lista negra bajo este criterio, para determinar si realmente sus publicaciones son despreciables para el análisis.

In [ ]:
# Obtenemos los tweets de estos usuarios, para revisarlos manualmente

# Ahorcarme
tweets_lisneg_ahorcarme <- publicaciones_ahorcarme_reducidas %>%
    filter(User %in% lista_negra)

# Colgarme
tweets_lisneg_colgarme <- publicaciones_colgarme_reducidas %>%
    filter(User %in% lista_negra)

In [ ]:
# Guardamos los tweets en un archivo

# Ahorcarme
escribir_dataframe(tweets_lisneg_ahorcarme, "./datos_limpios/datos_por_revisar/tweets_lisneg_ahorcarme.csv")
# Colgarme
escribir_dataframe(tweets_lisneg_colgarme, "./datos_limpios/datos_por_revisar/tweets_lisneg_colgarme.csv")

Luego de revisar los tweets, encontré un fragmento que hace alusión a una connotación sexual de la palabra concurrente. Por lo tanto, aquellos usuarios cuyas publicaciones que contienen la palabra "ahorcarme" serán añadidos a la lista negra.

Es importante considerar que, si bien emplean la palabra bajo un concepto sexual, puede no ser la única forma en la que la usan; de hecho, logré encontrar varios perfiles en los que hacen un uso sexual y violento del verbo. De tal manera, de este grupo de usuarios que tergiversan de esta forma la acción, únicamente se seleccionarán aquellos que emplean la palabra exclusivamente en un contexto sexual, es decir, en todos los tweets que les fueron recolectados que les pertenecen.

In [ ]:
# Fragmento
frags_frase = c(
    "ahorcame por dios ahorcame hasta el punto de que tus manos"
)

# Localizar los tweets que contengan la frase, y de ellos tomar los usuarios, para agregarlos a los que ya
# estaban listados, siempre y cuando todos los tweets del usuario contengan la frase

usuarios_frags_ahorcarme <- obtenerUsuariosRepetitivos(publicaciones_ahorcarme, frags_frase)

Otro son fragmentos de una canción, en los tweets con la palabra clave "colgarme"

Los fragmentos son los siguientes:

<blockquote>
    <ul>
        <li> Niña, dame una pestaña de tus ojos para colgarme de amor por ti </li>
        <li> Colgarme de cualquiera que le gusta trasnochar </li>
    </ul>
</blockquote>

In [ ]:
# Fragmentos
frags_frase = c(
    "Niña, dame una pestaña de tus ojos para colgarme de amor por ti",
    "Colgarme de cualquiera que le gusta trasnochar"
)
# Localizar los tweets que contengan la frase, y de ellos tomar los usuarios, para agregarlos a los que ya
# estaban listados.

usuarios_frags_colgarme <- obtenerUsuariosRepetitivos(publicaciones_colgarme, frags_frase)

Igualmente, se removerá cualquier uso de la llamada en el contexto de una conversación telefónica.

Por su parte, otros usos de la palabra que se refieran a una acción como "depender de" o "aprovecharse de" podrían requerir de otro tipo de servicios para ser procesados.

En [este notebook](./filtrado_publicaciones_por_contexto.ipynb) estaré trabajando en otros usos de la palabra a través de los servicios que ofrece Microsoft Azure. Indagaré en el uso de los servicios cognitivos o las integraciones con OpenAI para proveer un filtro de las publicaciones con un procesamiento del lenguaje más intuitivo y preciso.

In [ ]:
# No quitamos al usuario completamente porque el uso de la palabra "colgar" en una llamada no le excluye de usarla con
# intenciones autolesivas, por lo que nos limitamos a excluir esas publicaciones

palabras_contexto_telefonico <- c(
    "llamar",
    "marcar",
    "responder",
    "contestar"
)

# Demostrando el uso del paquete tm
raices_palabras_contexto_telefonico <- wordStem(palabras_contexto_telefonico, language = "spanish")
cat(raices_palabras_contexto_telefonico)

In [ ]:
# Eliminamos los tweets que contengan alguna conjugación de los verbos que indican que se habla de una llamada

tweets_colgarme_limpios <- excluir_tweets_segun_verbos(
    publicaciones_colgarme,
    palabras_contexto_telefonico,
    "spanish"
)

In [ ]:
# Escribir tweets para no depender de la sesión de R
escribir_dataframe(tweets_colgarme_limpios, "./datos_limpios/datos_por_revisar/tweets_colgarme_limpios.csv")

In [ ]:
# Escribir los mismos tweets, pero extrayendo aquellos cuyos usuarios aparecen con frecuencia

escribir_dataframe(
    obtener_publicaciones(tweets_colgarme_limpios, 5),
    "./datos_limpios/datos_por_revisar/tweets_colgarme_limpios_5_ocurrencias.csv"
)

In [ ]:
# Agregar los usuarios a la lista de usuarios

lista_negra <- juntar_usuarios(
    c(
        lista_negra,
        usuarios_frags_ahorcarme,
        usuarios_frags_colgarme
    )
)

In [ ]:
# Escribimos las listas de usuarios para no depender de la sesión de R
writeLines(usuarios_frags_ahorcarme, "./datos_limpios/datos_por_revisar/usuarios_frag_ahorcarme.txt")
writeLines(usuarios_frags_colgarme, "./datos_limpios/datos_por_revisar/usuarios_frag_colgarme.txt")

In [ ]:
# Escribir la lista negra en un archivo
writeLines(
    lista_negra,
    file = "./datos_limpios/datos_por_revisar/lista_negra.txt",
)

A partir de aquí, una vez que hallamos revisado la lista negra de usuarios, vamos a limpiar los conjuntos de datos para obtener únicamente la información útil.

In [ ]:
# Excluir a estos usuarios de los datasets, pero a partir de la lista negra de usuarios ya revisada

# Leer lista
lista_negra_revisada <- readLines("./datos_limpios/lista_negra_revisada.txt", encoding = "UTF-8")

In [ ]:
# Reescribir lista negra, para obtener sólo valores únicos
lista_negra_revisada <- unique(lista_negra_revisada) %>% sort()
writeLines(lista_negra_revisada, "./datos_limpios/lista_negra_revisada.txt")

In [ ]:
# Ahorcarme
publicaciones_ahorcarme <- retirar_tweets_usuarios(publicaciones_ahorcarme, lista_negra_revisada)

# Colgarme
publicaciones_colgarme <- retirar_tweets_usuarios(tweets_colgarme_limpios, lista_negra_revisada)

In [ ]:
# Guardamos los tweets en un archivo

# Ahorcarme
escribir_dataframe(publicaciones_ahorcarme, "./datos_limpios/ahorcarme_filtered.csv")

# Colgarme
escribir_dataframe(publicaciones_colgarme, "./datos_limpios/colgarme_filtered.csv")

### Dataset `necesito_ayuda_complete.csv`

En este dataset encontramos las siguientes stopwords:

- Estoy jugando #VenezuelaQuiz y necesito ayuda con esta imagen ¿La reconoces?
- ¡Necesito ayuda con mis árboles!
- Entra aquí
- Entra en
- _Usuarios con “1D” en su nombre (fans de One Direction solicitando ayuda para conseguir boletos)_
- One Direction
- sorteo
- concurso
- Sigan a
- Síganme
- rt
- retweet
- MG
- MG AL COMENTARIO
- bonus

Las usaremos para obtener a usuarios que utilicen esas frases en sus publicaciones, y los añadiremos a la lista negra.

In [ ]:
# Se lee el dataset

datos_necesito_ayuda <- read.csv(file = "./datos/twitter/necesito_ayuda_complete.csv")

In [ ]:
stopwords_necesito_ayuda <- c(
    "Estoy jugando #VenezuelaQuiz y necesito ayuda con esta imagen ¿La reconoces?",
    "¡Necesito ayuda con mis árboles!",
    "Entra aquí",
    "Entra en",
    "1D",
    "One Direction",
    "sorteo",
    "concurso",
    "Sigan a",
    "Siganme",
    "Síganme",
    "rt",
    "retweet",
    "retuit",
    "MG",
    "MG AL COMENTARIO",
    "bonus"
)

In [ ]:
usuarios_ls_negra_necesito_ayuda <- obtener_usuarios_segun_palabras(
    dataframe = datos_necesito_ayuda,
    palabras = stopwords_necesito_ayuda
)

datos_necesito_ayuda_limpios <- retirar_tweets_usuarios(
    datos_necesito_ayuda,
    usuarios_ls_negra_necesito_ayuda
)

In [ ]:
# Agregamos los usuarios a la lista negra

anexar_a_lista(usuarios_ls_negra_necesito_ayuda)

Además, retiraremos tweets que contengan enlaces

In [ ]:
datos_necesito_ayuda_limpios <- retirar_tweets_con_enlaces(datos_necesito_ayuda_limpios)

In [ ]:
escribir_dataframe(
    datos_necesito_ayuda_limpios,
    "./datos_limpios/twitter/necesito_ayuda_cleaned.csv"
)

Encontramos, además, propaganda política en los tweets de los usuarios. A menudo, etiquetan al (al año de 2023) presidente de Venezuela, Nicolás Maduro, en sus publicaciones.

No obstante, si revisamos estas cuentas podremos percatarnos de que en sus publicaciones existen peticiones por recursos, pues son personas en situación de calle, desempleo, o que no tienen acceso a servicios básicos. Por ello, se tomarán estos usuarios y se colocaran en una "lista gris", para que puedan ser examinados después por especialistas y determinar si su situación les puede suscitar conductas depresivas.

### Dataset `perdon_por_todo_complete.csv`

En este dataset encontramos estas frases que podrían ser de interés excluir:

In [ ]:
expresiones_perdon_por_todo <- c(
    "Perdón por ser fiel, entregar todo de mí, quererte como a nadie",
    "Gracias por enseñarme lo que debo mejorar",
    "al querer ser perfecto se equivoca todo el mundo",
    "Ya no queda valor para mirarnos de nuevo y pedirnos",
    "Gracias por todo y perdón por ser tan básico",
    "Corazón, perdón por todo el daño"
)

datos_perdon_por_todo <- read.csv(file = "./datos/twitter/perdon_por_todo_complete_fixed.csv")

In [ ]:
usuarios_ls_negra_perdon <- obtenerUsuariosRepetitivos(
    dataset = datos_perdon_por_todo,
    textosClave = expresiones_perdon_por_todo
)

In [ ]:
usuarios_ls_negra_perdon[1:20]

In [ ]:
# Agregamos los usuarios a la lista negra

anexar_a_lista(usuarios_ls_negra_perdon)

In [ ]:
# Retiramos los usuarios del dataset y los guardamos en un archivo
datos_perdon_por_todo_limpios <- retirar_tweets_usuarios(
    datos_perdon_por_todo,
    usuarios_ls_negra_perdon
)

# Retiramos los tweets con URLs

datos_perdon_por_todo_limpios <- retirar_tweets_con_enlaces(
    datos_perdon_por_todo_limpios
)

escribir_dataframe(
    datos_perdon_por_todo_limpios,
    "./datos_limpios/twitter/perdon_por_todo_cleaned.csv"
)

### Dataset `suicida_complete.csv`

In [17]:
# Se leen los datos

datos_suicida <- read.csv(file = "./datos/twitter/suicida_complete.csv")

In [ ]:
# De los usuarios que tienen la palabra “suicida” en el nombre, se crea un dataset aparte y estos se quitan del dataset,
# pues no necesariamente llevan la palabra en el tweet

usuarios_nombre_llevan_suicida <- datos_suicida %>%
    filter(grepl("suici", User, ignore.case = TRUE)) %>%
    distinct(User) %>%
    pull(User)

In [ ]:
# Se guardan los datos de estos usuarios en un archivo

datos_usuarios_llevan_suicida <- datos_suicida %>%
    filter(User %in% usuarios_nombre_llevan_suicida)

escribir_dataframe(
    datos_usuarios_llevan_suicida,
    "./datos_limpios/twitter/usuarios_nombre_llevan_suicida_complete.csv"
)

Por su parte, debido a que tenemos como objetivos sitios de noticias también y estos están siendo reportados en un archivo adicional, vamos a reconocerlos y registrarlos en la lista negra y este archivo.

In [54]:
patrones_noticieros <- 
c(
    "noticia",
    "period",
    "noti",
    "news",
    "nws",
    "www",
    "deport",
    "juegos",
    "diario",
    "futbol",
    "canal",
    "cronica",
    "espectaculo",
    "radio"
)

In [ ]:
usuarios_noticieros <- datos_suicida %>%
    filter(grepl(paste(patrones_noticieros, collapse = "|"), User, ignore.case = TRUE)) %>%
    distinct(User) %>%
    pull(User) 

In [20]:
print(paste0("Usuarios noticieros: ", length(usuarios_noticieros)))
usuarios_noticieros[1:20]

[1] "Usuarios noticieros: 4740"


[1] "Noticias_Ec"     "UNoticias"       "NoticiasTribuna" "NoticiasRCN"    
 [5] "SuNoticiero"     "noticiasLGTB"    "PUTOnews"        "AmaneceNews"    
 [9] "orpnoticias"     "PeriodicoZocalo" "Hera_Noticias"   "ControlNoticias"
[13] "NotiEspartano"   "NotiSeguridadMX" "AztecaNoticias"  "QroNotiLocales" 
[17] "Venezolanonews"  "seguridad_news"  "24noticiareport" "Noti_Momento"

In [31]:
# Agregamos los usuarios a la lista negra

anexar_a_lista(usuarios_noticieros)

# Agregamos además los usuarios al archivo de noticieros
anexar_a_lista(
    usuarios_noticieros,
    ruta = "./datos_limpios/lista_pags_noticias.txt"
)

Ahora, comenzamos a elaborar la lista negra para este conjunto de datos.

In [26]:
# Se marcan en la lista negra usuarios con publicaciones que contengan las siguientes palabras/expresiones/fragmentos:
expresiones_suicida <-
c(
    "terrorista",
    "ataque",
    "atacante",
    "piloto",
    "aerol", # Por aerolínea
    "avión",
    "avion",
    "bomba",
    "atentado",
    "masacre",
    "RT",
    "noticias",
    "noticia",
    "noti",
    "news",
    "presunto",
    "polic", # Por policía
    "ONU",
    "OTAN",
    "iran",
    "irán",
    "pakist" # Por Pakistán
)

Las expresiones "se suicida" y "escuadrón" también son usadas por algunas personas con su cuenta personal.

No obstante, sus publicaciones típicamente no superan las 10. Esto se tomará en cuenta, para colocar en la lista negra a los usuarios siempre y cuando rebasen este límite.

In [47]:
# Lista negra base
usuarios_ls_negra_suicida_base <- obtener_usuarios_segun_palabras(
    dataframe = datos_suicida,
    palabras = expresiones_suicida
)

In [48]:
# Se toman estos usuarios y se revisa que sus publicaciones rebasen las 10. Estos irán a la lista negra

exp_suicida_especiales <- c(
    "escuadr",
    "se suicida"
)

usuarios_ls_negra_suicida_escuad_sesuic <- obtener_publicaciones(
        dataframe = datos_suicida,
        pub_minimas = 10
    ) %>%
    filter(grepl(paste(exp_suicida_especiales, collapse = "|"), Content, ignore.case = TRUE)) %>%
    distinct(User) %>%
    pull(User)    

In [49]:
usuarios_ls_negra_suicida <- juntar_usuarios(
    c(
        usuarios_ls_negra_suicida_base,
        usuarios_ls_negra_suicida_escuad_sesuic
    )
)

usuarios_ls_negra_suicida[1:20]

[1] "foroson123"      "murillocasanova" "cb24tv"          "PuebladeHoy"    
 [5] "laguaiqueri1"    "MauroECast"      "CUMANAINFORMA"   "Guillermodelao" 
 [9] "nacotemx"        "TresCanicas"     "Manuelglezp"     "Noticias_Ec"    
[13] "patyvargaas"     "lavozdeturmero"  "algonzalez72"    "juanlol0"       
[17] "marce_godoy1"    "gudnieta"        "azorindorian"    "comuvm_rdd"

In [50]:
# Agregamos los usuarios a la lista negra
anexar_a_lista(usuarios_ls_negra_suicida)

In [51]:
# Retiramos los usuarios del dataset

datos_suicida_limpios <- retirar_tweets_usuarios(
    datos_suicida,
    usuarios_ls_negra_suicida
)

In [52]:
# Retiramos los tweets con URLs también

datos_suicida_limpios <- datos_suicida_limpios %>%
    retirar_tweets_con_enlaces()

In [53]:
# Escribimos los datos limpios

escribir_dataframe(
    datos_suicida_limpios,
    "./datos_limpios/twitter/suicida_cleaned.csv"
)

Warning message in write.csv(dataframe, file = ruta_archivo, row.names = FALSE, :
"attempt to set 'col.names' ignored"


### datasets `queria_estar_muerto_cleaned.csv` y `queria_morir_cleaned.csv`

Para estos conjuntos de datos únicamente retiraremos los enlaces. Por otro lado, el filtrado de los usuarios se hizo manualmente para el conjunto de datos `queria_estar_muerto`, mientras que para el otro se definirá una lista negra y sí se retirarán los usuarios.

In [43]:
datos_queria_estar_muerto_limpios <- read.csv(file = "./datos_limpios/twitter/queria_estar_muerto_cleaned.csv") %>%
    retirar_tweets_con_enlaces()

escribir_dataframe(
    datos_queria_estar_muerto_limpios,
    "./datos_limpios/twitter/queria_estar_muerto_cleaned.csv"
)

Warning message in write.csv(dataframe, file = ruta_archivo, row.names = FALSE, :
"attempt to set 'col.names' ignored"


In [55]:
usuarios_ls_negra_queria_morir <- c(
    "_LaPista_",
    "_Showzzy",
    "24HDeportesTVN",
    "A_bran_es",
    "abc_familia",
    "AcentoVNoticias",
    "ActualidadRT",
    "alexascarpita",
    "alikton",
    "AnaRMarti",
    "AndresCardeenas",
    "aylinj1D",
    "bekiaes",
    "BogotaET",
    "brayan_insur_12",
    "cactus24noticia",
    "canaltn8",
    "capdevielleja",
    "CaraotaDigital",
    "CCNesnoticias",
    "Citytv",
    "Con_Sentimiento",
    "ConservadorMX",
    "contrapuntovzla",
    "CorreodelCaroni",
    "CorreodeOaxaca",
    "CronicaUno",
    "Cuida_Emocional",
    "CulturaColectiv",
    "Danirobonita",
    "demamasdepapas",
    "DirectionerB1D",
    "DolarToday",
    "DoralHoy",
    "e_consulta",
    "Ecoteuve",
    "EfectoNaim",
    "El_Universal_Mx",
    "el_pais",
    "elandivar",
    "elespectador",
    "elias0998",
    "eljuntaletrass",
    "elpaismexico",
    "ElPitazoTV",
    "eltelevisero",
    "ELTIEMPO",
    "EncarnaSanchezJ",
    "enlazadot",
    "EP_Mundo",
    "EresCurioso",
    "escuelalemat",
    "EvaRiquelme3",
    "exitoina",
    "fabiomb",
    "Franz_Strada",
    "FrenteFantasma",
    "Gabyg1233",
    "gpascual96",
    "GuzmanJoseVicen",
    "HayQueSaberlo",
    "HipcritasNoWD",
    "historiasqlaten",
    "HiwatarixValkov",
    "hoyextremadura",
    "HoyTeInformo",
    "infobae",
    "infobaeamerica",
    "InfoVelozCom",
    "ivan_ramibal",
    "JosiriOsorio",
    "juaneinacio",
    "la_patilla",
    "LaCronicaDeHoy",
    "LAFAYETTERICO",
    "lasillarota",
    "Leandro68455164",
    "legs3535",
    "los_replicantes",
    "lucasecastillo",
    "m1espectaculos",
    "MicaelaRomero_1",
    "MiLugarFavorit2",
    "NicaNws",
    "NicoKontreras",
    "NubeyChloe",
    "Pajaropolitico",
    "pame_hunter",
    "PaparazziRevis",
    "paulinotech",
    "pauliqw591",
    "PeterKrasno",
    "publimetro_dep",
    "PublimetroChile",
    "punto7osorno",
    "Punto7ptomontt",
    "punto7temuco",
    "QuePasaVenado",
    "RadioImpacTotal",
    "RedRadioVe",
    "Reportajes_org",
    "RunRunesWeb",
    "sabiastuque_",
    "SadujDark",
    "safal4776033",
    "segurosybanca",
    "SinEmbargoMX",
    "SoyModaNet",
    "superconfirmado",
    "SwaggieVane",
    "tar88t",
    "TeleSaltillo",
    "teleSURtv",
    "thecliniccl",
    "TVNotasmx",
    "un_dato",
    "valeriecortesnh",
    "VenadoTuerto140",
    "VenezolanoPres_",
    "VestidoFiesta",
    "yakibracamontes"
)

# Ag"regamos los usuarios a la lista negra",
anexar_a_lista(usuarios_ls_negra_queria_morir)

In [56]:
datos_queria_morir <- read.csv(file = "./datos/twitter/queria_morir_complete.csv")

In [57]:
usuarios_noticieros_queria_morir <- datos_queria_morir %>%
    filter(grepl(paste(patrones_noticieros, collapse = "|"), User, ignore.case = TRUE)) %>%
    distinct(User) %>%
    pull(User)

# Lista negra base
anexar_a_lista(usuarios_noticieros_queria_morir)
# Agregamos además los usuarios al archivo de noticieros
anexar_a_lista(
    usuarios_noticieros_queria_morir,
    ruta = "./datos_limpios/lista_pags_noticias.txt"
)

In [59]:
usuarios_ls_negra_queria_morir_join <- juntar_usuarios(

    c(
        usuarios_ls_negra_queria_morir,
        usuarios_noticieros_queria_morir,
        obtenerUsuariosRepetitivos(
            dataset = datos_queria_morir,
            textosClave = c(
                "Tu mirada fue el espejo , donde quería ver nuestro reflejo",
                "de amor, para que supieras cómo y cuánto te quería",
                "matarme a besos, yo quería morir en tus labios"
            )
        )
    )
)

datos_queria_morir_limpios <- datos_queria_morir %>%
    retirar_tweets_usuarios(usuarios_ls_negra_queria_morir_join) %>%
    retirar_tweets_con_enlaces()

In [60]:
escribir_dataframe(
    datos_queria_morir,
    ruta_archivo = './datos_limpios/twitter/queria_morir_cleaned.csv'
)

Warning message in write.csv(dataframe, file = ruta_archivo, row.names = FALSE, :
"attempt to set 'col.names' ignored"


#### datasets `quiero_estar_muerto_complete.csv` y `quisiera_estar_muerto.csv`

In [17]:
datos_quiero_estar_muerto <- read.csv(file = "./datos/twitter/quiero_estar_muerto_complete.csv")

In [21]:
usuarios_ls_quiero_estar_muerto <- c(
    "carolinad_bot",
    "botpoemas",
    "rockolaperu",
    "Samyurin",
    "wxzm"
)

usuarios_frags_quiero_estar_muerto <- obtenerUsuariosRepetitivos(
    dataset = datos_quiero_estar_muerto,
    textosClave = c(
        "dile que estoy muerto, dile que estoy seco",
        "dile que estoy seco que quiero estar con ella",
        "quiero soñar con los ojos abiertos, quiero sentir, sentir el fuego",
        "me tienes muerto, sin rumbo como un vagabundo",
        "y siempre quiero estar muerto, para seguir con mi boca enredada en tus cabellos",
        "muerto de la risa, así es como quiero estar",
        "Yo te quiero con el alma, que el alma nunca muere"
    ),
    ignorarSignosPunt = TRUE # El dataset es chico y podemos permitirnos la búsqueda de repetición
                             # con frases con o sin signos de puntuación
)

datos_quiero_estar_muerto_limpios <- datos_quiero_estar_muerto %>%
    retirar_tweets_usuarios(usuarios_ls_quiero_estar_muerto) %>%
    retirar_tweets_con_enlaces()

In [22]:
usuarios_ls_quiero_estar_muerto[1:20]

[1] "carolinad_bot" "botpoemas"     "rockolaperu"   "Samyurin"     
 [5] "wxzm"          NA              NA              NA             
 [9] NA              NA              NA              NA             
[13] NA              NA              NA              NA             
[17] NA              NA              NA              NA

In [23]:
anexar_a_lista(usuarios_ls_quiero_estar_muerto)

In [24]:
escribir_dataframe(
    datos_quiero_estar_muerto_limpios,
    ruta_archivo = './datos_limpios/twitter/quiero_estar_muerto_cleaned.csv'
)

Warning message in write.csv(dataframe, file = ruta_archivo, row.names = FALSE, :
"attempt to set 'col.names' ignored"


In [25]:
datos_quisiera_estar_muerto <- read.csv(file = "./datos/twitter/quisiera_estar_muerto_complete.csv")

In [26]:
usuarios_ls_quisiera_estar_muerto <- c(
    "escribircancion",
    "nirumi80"
)

usuarios_frags_qusiera_estar_muerto <- obtenerUsuariosRepetitivos(
    dataset = datos_quisiera_estar_muerto,
    textosClave = c(
        "no quisiera estar muerto, al menos no tanto"
    ),
    ignorarSignosPunt = TRUE # El dataset es chico y podemos permitirnos la búsqueda de repetición
                             # con frases con o sin signos de puntuación
)

datos_quisiera_estar_muerto_limpios <- datos_quisiera_estar_muerto %>%
    retirar_tweets_usuarios(usuarios_ls_quisiera_estar_muerto) %>%
    retirar_tweets_con_enlaces()

In [27]:
usuarios_ls_quisiera_estar_muerto[1:20]

[1] "escribircancion" "nirumi80"        NA                NA               
 [5] NA                NA                NA                NA               
 [9] NA                NA                NA                NA               
[13] NA                NA                NA                NA               
[17] NA                NA                NA                NA

In [28]:
anexar_a_lista(usuarios_ls_quisiera_estar_muerto)

In [29]:
escribir_dataframe(
    datos_quisiera_estar_muerto_limpios,
    ruta_archivo = './datos_limpios/twitter/quisiera_estar_muerto_cleaned.csv'
)

Warning message in write.csv(dataframe, file = ruta_archivo, row.names = FALSE, :
"attempt to set 'col.names' ignored"


### dataset `matarme_complete.csv`

Por último, podemos también ordenar la lista negra, para que cuando esta se requiera, se pueda acceder a sus valores más eficientemente a través de un algoritmo de búsqueda binaria.

In [30]:
lista_negra_final <- cargar_lista_negra() %>% sort()

escribir_lista_negra(
    lista_negra_final#,
    # retirar_duplicados = TRUE
)

In [31]:
# Ordenamos la lista de páginas de noticias
usuarios_noticieros <- readLines("./datos_limpios/lista_pags_noticias.txt") %>%
    # unique() %>%
    sort()

# Se guardan los datos de estos usuarios en un archivo
writeLines(usuarios_noticieros, "./datos_limpios/lista_pags_noticias.txt")

### dataset `suicidal_corpus_complete_translated.csv`

Este conjunto de datos incluyen textos clasificados como aquellos que denotan riesgo suicida y aquellos que no.

Para que el corpus de datos sea útil, el director del proyecto se solicitó que cada texto tenga, cuanto menos, 5 palabras, y que no se repitan textos.

In [51]:
dataset_corpus_suicida <- read.csv(file = "./datos/otros/suicidal_corpus_complete_translated.csv")

filas_prev <- nrow(dataset_corpus_suicida)

Afortunadamente, también se cuenta con el dataset antes de realizar la traducción de los textos que se van a procesar. Por lo tanto, debido a que este dataset será sometido a una revisión manual de las traducciones por parte del equipo, vamos a tomar el texto original y traducido para que las correcciones sean más precisas.

In [59]:
dataset_corpus_suicida_ingles <- read.csv(file = "./datos/otros/suicidal_corpus_complete_original.csv")

# Nos aseguramos de que las columnas que contienen la clasificación de los textos sean iguales, para que, al momento
# de unir los datasets, no corrompamos la información

print("Las columnas de clasificación son iguales:")
identical(
    dataset_corpus_suicida$cls,
    dataset_corpus_suicida_ingles$cls
)

[1] "Las columnas de clasificación son iguales:"


[1] TRUE

In [60]:
dataset_corpus_suicida <- data.frame(
    text_original = dataset_corpus_suicida_ingles$text,
    text_translated = dataset_corpus_suicida$text,
    cls = dataset_corpus_suicida$cls
)

In [65]:
dataset_corpus_suicida_reducido <- dataset_corpus_suicida %>%
    filter(str_count(text_translated, "\\w+") >= 5) %>%
    distinct(text_translated, .keep_all = TRUE)

escribir_dataframe(
    dataset_corpus_suicida_reducido,
    ruta_archivo = './datos_limpios/datos_por_revisar/otros/suicidal_corpus_reduced_joined.csv'
)

filas_post <- nrow(dataset_corpus_suicida_reducido)

filas_reducidas <- filas_prev - filas_post

print(paste0("Se redujeron ", filas_reducidas, " filas"))

Warning message in write.csv(dataframe, file = ruta_archivo, row.names = FALSE, :
"attempt to set 'col.names' ignored"


[1] "Se redujeron 470 filas"
